In [31]:
import subprocess
import random
import fastaparser
import os

import pandas as pd

from src.dna import *

We will need to modify a sequences using the tool, then compare it against the background.

In [2]:
random.seed(867530)

min = 8
max = 20

sequences = [random_dna(i) for i in range(min, max+1)]

sequences

['CTTTAACA',
 'CATCGGACC',
 'TGGGGGGGCG',
 'TCCGTGCTGTA',
 'CGGGGCTAGTTC',
 'CTACGCCAATGTA',
 'GGTAACCGTATGTT',
 'ACTCTACGCAGCGCA',
 'CAAACTCGTTGGAACA',
 'TGAACATTTCGGTTATA',
 'CGCGCGAAGAACTTGGCA',
 'TCGATGAATTACGGAAACG',
 'ACCTATCGATCCAACCTAGG']

In [3]:
filepath = "data/"
filename = "U00096.3.fasta"

for seq in sequences:

    fasta_obj = read_fasta(filepath+filename)
    fasta_header = fasta_obj["header"]
    fasta_seq = fasta_obj["sequence"]

    new_sequence = insert_sequence(seq, fasta_seq)

    seq_id = str(len(seq)) + "_" + seq

    new_filename = filepath + "modified/" + seq_id + "_" + filename

    with open(new_filename, "w") as fasta_file:
        fasta_seq_mod = fastaparser.FastaSequence(
            sequence = new_sequence,
            # id = seq_id,
            description = filename + " with " + seq + "introduced"
        )

In [4]:
f = read_fasta(filename = "data/U00096.3.fasta")

type(f)

dict

In [20]:
mod_dirs = os.listdir("homer-results/")

mod_dirs

['motifs_CTTTAACA',
 'motifs_CATCGGACC',
 'motifs_TGGGGGGGCG',
 'motifs_TCCGTGCTGTA',
 'motifs_CGGGGCTAGTTC',
 'motifs_CTACGCCAATGTA',
 'motifs_GGTAACCGTATGTT',
 'motifs_ACTCTACGCAGCGCA',
 'motifs_CAAACTCGTTGGAACA',
 'motifs_TGAACATTTCGGTTATA',
 'motifs_CGCGCGAAGAACTTGGCA',
 'motifs_TCGATGAATTACGGAAACG',
 'motifs_ACCTATCGATCCAACCTAGG']

## Random DNA Does not appear to fool it in this experiment
    > Limitation: degenerate codons

In [45]:
results_dict = {}

for mod_file in mod_dirs:
    insert_sequence = mod_file.split("_")[1]
    outdirs = os.listdir("homer-results/" + mod_file)
    for f in outdirs:
        if "motif.2.tmp" in f:
            motif_data = pd.read_csv("homer-results/" + mod_file + "/" + f, sep = "\t")
            motifs = motif_data["Consensus"]
            found = False
            for m in motifs:
                if m == insert_sequence:
                    found = True
            results_dict[insert_sequence] = found
results_dict     

{'CTTTAACA': False,
 'CATCGGACC': False,
 'TGGGGGGGCG': False,
 'TCCGTGCTGTA': False,
 'CGGGGCTAGTTC': False,
 'CTACGCCAATGTA': False,
 'GGTAACCGTATGTT': False,
 'ACTCTACGCAGCGCA': False,
 'CAAACTCGTTGGAACA': False,
 'TGAACATTTCGGTTATA': False,
 'CGCGCGAAGAACTTGGCA': False,
 'TCGATGAATTACGGAAACG': False,
 'ACCTATCGATCCAACCTAGG': False}

In [7]:
# example motifs
# findMotifs.pl targets.fa fasta motifResults/ -fasta background.fa
# for f in mod_files:

#     subprocess.call([
#         "findMotifs.pl",
#         "data/U00096.3.fasta",
#         "fasta",
#         "homer-results/motifs_" + f.split("_")[1],
#         "-fasta",
#         "data/modified/" + f
#     ])